<a href="https://colab.research.google.com/github/Jaydenzk/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [91]:
### YOUR CODE STARTS HERE

import pandas as pd
import numpy as np
from google.colab import files
files.upload()

Saving house-votes-84.data to house-votes-84 (1).data


{'house-votes-84.data': b'republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y\nrepublican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?\ndemocrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n\ndemocrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y\ndemocrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y\ndemocrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y\ndemocrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y\nrepublican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y\nrepublican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y\ndemocrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?\nrepublican,n,y,n,y,y,n,n,n,n,n,?,?,y,y,n,n\nrepublican,n,y,n,y,y,y,n,n,n,n,y,?,y,y,?,?\ndemocrat,n,y,y,n,n,n,y,y,y,n,n,n,y,n,?,?\ndemocrat,y,y,y,n,n,y,y,y,?,y,y,?,n,n,y,?\nrepublican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?\nrepublican,n,y,n,y,y,y,n,n,n,y,n,y,y,?,n,?\ndemocrat,y,n,y,n,n,y,n,y,?,y,y,y,?,n,n,y\ndemocrat,y,?,y,n,n,n,y,y,y,n,n,n,y,n,y,y\nrepublican,n,y,n,y,y,y,n,n,n,n,n,?,y,y,n,n\ndemocrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y\ndemocrat,y,y,y,n,n,?,y,y,n,n,y,n,n,n,y,y\ndemocrat,y,y,y,n,n,n,y,y,y,n,n,n,?,?,y,y\ndemocrat,y,?,y,n,n,n,y,y,y,n,n,?

In [127]:
## load data
df = pd.read_csv('house-votes-84.data')
df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
df.columns = ["party", "handicapped_infants", "water_project_cost_sharing", "adoption_of_the_budget_resolution", "physician_fee_freeze", "el_salvador_aid", "religious_groups_in_schools", "anti_satellite_test_ban", "aid_to_nicaraguan_contras", "mx_missile", "immigration", "synfuels_corporation_cutback", "education_spending", "superfund_right_to_sue", "crime", "duty_free_exports", "export_administration_act_south_africa"]

In [129]:
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [0]:
df.replace(["y","n"],["1","0"], inplace = True)

In [131]:
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,?
1,democrat,?,1,1,?,1,1,0,0,0,0,1,0,1,1,0,0
2,democrat,0,1,1,0,?,1,0,0,0,0,1,0,1,0,0,1
3,democrat,1,1,1,0,1,1,0,0,0,0,1,?,1,1,1,1
4,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1


In [0]:
df.replace('?', np.NaN, inplace = True)

In [133]:
df.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,NaN
1,democrat,NaN,1,1,NaN,1,1,0,0,0,0,1,0,1,1,0,0
2,democrat,0,1,1,0,NaN,1,0,0,0,0,1,0,1,0,0,1
3,democrat,1,1,1,0,1,1,0,0,0,0,1,NaN,1,1,1,1
4,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1


In [134]:
df.describe()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
count,434,422,386,423,423,419,423,420,419,412,427,414,403,409,417,406,330
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,0,1,1,0,1,1,1,1,1,1,0,0,1,1,0,1
freq,267,235,194,253,247,211,271,239,242,207,215,264,233,208,247,232,268


In [135]:
rep = df[df['party'] =='republican']
rep.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,NaN
6,republican,0,1,0,1,1,1,0,0,0,0,0,0,1,1,NaN,1
7,republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,1
9,republican,0,1,0,1,1,0,0,0,0,0,NaN,NaN,1,1,0,0
10,republican,0,1,0,1,1,1,0,0,0,0,1,NaN,1,1,NaN,NaN


In [136]:
dem = df[df['party'] =='democrat']
dem.head()

,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
1,democrat,NaN,1,1,NaN,1,1,0,0,0,0,1,0,1,1,0,0
2,democrat,0,1,1,0,NaN,1,0,0,0,0,1,0,1,0,0,1
3,democrat,1,1,1,0,1,1,0,0,0,0,1,NaN,1,1,1,1
4,democrat,0,1,1,0,1,1,0,0,0,0,0,0,1,1,1,1
5,democrat,0,1,0,1,1,1,0,0,0,0,0,0,NaN,1,1,1


In [137]:
dem['water_project_cost_sharing'].value_counts(dropna=False)

1      120
0      119
NaN     28
Name: water_project_cost_sharing, dtype: int64

In [138]:
dem['water_project_cost_sharing'].mean()

TypeError: ignored

In [0]:
from scipy.stats import ttest_1samp

In [48]:
ttest_1samp(dem['water-project-cost-sharing'], .5, nan_policy='omit')

TypeError: ignored

In [87]:
ttest_1samp(dem['water-project-cost-sharing'], 0, nan_policy='omit')

TypeError: ignored

In [88]:
ttest_1samp(dem['water-project-cost-sharing'], 1, nan_policy='omit')

TypeError: ignored

In [89]:
ttest_1samp(dem['water-project-cost-sharing'], .6, nan_policy='omit')

TypeError: ignored